## Load data
source: https://www.kaggle.com/datasets/samuelcortinhas/cats-and-dogs-image-classification

In [ ]:
import os
import torch
from torchvision import transforms
from PIL import Image

INPUT_SIZE = (128, 128)

images_dir = 'datasets/dogs_cats/train'

transform = transforms.Compose([
    transforms.Resize(INPUT_SIZE),
    transforms.ToTensor()
])

CLASSES = 2

def load_dataset():
    data = []
    labels = []

    for image_file in os.listdir(images_dir):
        try:
            # Load and process image
            img_path = os.path.join(images_dir, image_file)
            image = Image.open(img_path).convert('RGB')
            image = transform(image)
            data.append(image)
            labels.append(0 if image_file.startswith('cat') else 1)

        except Exception as e:
            print(f"Failed to process {img_path}: {e}")
    
    return torch.stack(data), torch.tensor(labels)

# Load the data
data, labels = load_dataset()

## Normalize and visualize

In [ ]:
import matplotlib.pyplot as plt

print(data.shape)
print(labels.shape)

# data = data * 2 - 1


print(data)
print(labels)


plt.hist(labels, bins=range(CLASSES+1), align='left', rwidth=0.9);

In [ ]:
from augmentation import show_random

show_random(data, labels)

## Create model

In [ ]:
import residual
import torch.nn as nn

class CNNv2(nn.Module):
    def __init__(self, classes):
        super(CNNv2, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=13, stride=2, dilation=3, padding=5)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=7, stride=1, padding=3)
        self.layer1 = residual.ResidualBlock(64, 64)
        self.layer2 = residual.ResidualBlock(64, 128, stride=2,
            downsample=nn.Sequential(nn.Conv2d(64, 128, kernel_size=1, stride=2), nn.BatchNorm2d(128)))
        self.layer3 = residual.ResidualBlock(128, 256, stride=2,
            downsample=nn.Sequential(nn.Conv2d(128, 256, kernel_size=1, stride=2), nn.BatchNorm2d(256)))
        # self.layer4 = residual.ResidualBlock(256, 512, stride=2,
        #     downsample=nn.Sequential(nn.Conv2d(256, 512, kernel_size=1, stride=2), nn.BatchNorm2d(512)))
        self.conv2 = nn.Conv2d(256, 512, kernel_size=4, stride=1, dilation=1, padding=2)
        self.bn2 = nn.BatchNorm2d(512)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, classes)
        self.dropout = nn.Dropout(0.3)


    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.dropout(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.dropout(x)
        x = self.layer2(x)
        x = self.dropout(x)
        x = self.layer3(x)
        x = self.dropout(x)
        # x = self.layer4(x)
        x = self.dropout(x)
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x


raw_model = CNNv2(CLASSES)

raw_model.apply(residual.init_weights)

print(raw_model)

## Create dataset

In [ ]:
from augmentation import apply_random_filters
from torch.utils.data import DataLoader, TensorDataset, random_split
from torch.utils.data import Dataset

BATCH = 8
TRAIN_FRATION = 0.8 

full_dataset = TensorDataset(data, labels)

# Splitting the dataset into training and validation datasets
train_size = int(TRAIN_FRATION * len(full_dataset))
val_size = len(full_dataset) - train_size

train_data, val_data = random_split(full_dataset, [train_size, val_size])

def transform_wrapper(image):
    return apply_random_filters(image, probabilities=[0.3, 0.2, 0.3, 0.2])


class CustomDataset(Dataset):
    def __init__(self, subset, transform=None):
        self.subset = subset
        self.transform = transform

    def __len__(self):
        return len(self.subset)

    def __getitem__(self, idx):
        image, label = self.subset[idx]
        if self.transform:
            image = self.transform(image)
        return image, label


train_dataset = CustomDataset(train_data, transform=transform_wrapper)
train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH, shuffle=True)

test_dataset = CustomDataset(val_data)  # No transformation for test data
test_loader = DataLoader(dataset=test_dataset, batch_size=BATCH, shuffle=False)

## Train

In [ ]:
from fit import fit
import torch.nn as nn
import torch.optim as optim 

LR = 0.0007
EPOCH = 20

# Define Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(raw_model.parameters(), lr=LR)

fit(raw_model, train_loader, test_loader,
    criterion, optimizer, EPOCH, CLASSES, 'backup/dogs_cats/residual_cnn_raw_model_1')

## Create model using pretrained weights

In [ ]:
import residual


class TransferCNN(nn.Module):
    def __init__(self, pretrained_model):
        super(TransferCNN, self).__init__()
        # Copy layers from the pre-trained model
        self.conv1 = pretrained_model.conv1
        self.bn1 = pretrained_model.bn1
        self.relu = pretrained_model.relu
        self.maxpool = pretrained_model.maxpool
        self.layer1 = pretrained_model.layer1
        self.layer2 = pretrained_model.layer2
        self.layer3 = pretrained_model.layer3
        # self.layer4 = pretrained_model.layer4

        # self.new_layer4 = residual.ResidualBlock(256, 512, stride=2,
        #     downsample=nn.Sequential(nn.Conv2d(256, 512, kernel_size=1, stride=2), nn.BatchNorm2d(512)))
        self.new_conv = nn.Conv2d(256, 512, kernel_size=4, stride=1, padding=1)
        self.new_bn = nn.BatchNorm2d(512)
        self.new_avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.new_fc = nn.Linear(512, CLASSES)  # Adjust num_new_classes as needed

    def forward(self, x):
        # Use the same forward method until layer4
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        # x = self.layer4(x)

        # New layers for the new task
        x = self.relu(self.new_bn(self.new_conv(x)))
        x = self.new_avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.new_fc(x)
        return x

In [ ]:
pretrained_model = residual.CNNv1(10)
pretrained_model.load_state_dict(torch.load('backup/cifar_10/model_1e5v2_epoch_0'))
transfer_model = TransferCNN(pretrained_model)

# transfer_model.apply(init_weights)
# transfer_model.new_layer4.apply(residual.init_weights)
transfer_model.new_conv.apply(residual.init_weights)
transfer_model.new_bn.apply(residual.init_weights)  # Only if you want to initialize BatchNorm layers
transfer_model.new_fc.apply(residual.init_weights)

## Train model

In [ ]:

LR = 0.0003
EPOCH = 16 

# Define Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(raw_model.parameters(), lr=LR)

fit(pretrained_model, train_loader, test_loader,
    criterion, optimizer, EPOCH, CLASSES, 'backup/dogs_cats/residual_cnn_transfer_model_1')